# EDA and Initial Model Visualization

In [10]:
""" Notebook Setup/Imports
"""

# Project path setup
from pathlib import Path
project_dir = Path(__name__).resolve().parents[1]

# Library import
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.figure_factory as ff

# Print strings as markdown
from IPython.display import Markdown

# Data variables from previous notebook
dataset = pd.read_pickle(Path.joinpath(project_dir, "data/processed/1_data.pickle"))
data_dictionary = pd.read_pickle(Path.joinpath(project_dir, "data/processed/1_data_dictionary.pickle"))

## Plot USDA Model

In [12]:
fips = dataset.index.to_list()
values = [ 1 if x else 0 for x in dataset['USDA Model']]

fig = ff.create_choropleth(fips=fips, values=values)
py.iplot(fig, filename='USDA Initial Model', sort=False)

The draw time for this plot will be slow for all clients.
